#**Requirement 2: Nearest Neighbor Search and Duplicate Detection**

## **Question 2α:​ De-Duplication with Locality Sensitive Hashing**

In [1]:
import pandas as pd
import numpy as np
from time import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('drive/My Drive/Fall-2020/Big Data/Question2/corpusTrain.csv')
train

,Id,Content
0,0,How many people are going towards using phones...
1,1,What audio format should I use for getting aud...
2,2,What is the corporate culture like at Edwards ...
3,3,What is the best barbecue in Kansas City?\n
4,4,"""Can I combine the output of two bolts to one ..."
...,...,...
531985,531985,Why is SEO important?\n
531986,531986,Who is the best employer for Robotic Process a...
531987,531987,Is it possible to cure the holes caused by pim...
531988,531988,How many employees does Infosys have?\n


In [4]:
test = pd.read_csv('drive/My Drive/Fall-2020/Big Data/Question2/corpusTest.csv')
test

,Id,Content
0,0,How do I get good marks in college?\n
1,1,Can an android app use SMS only to communicate...
2,2,What small detail from an Indian movie do you ...
3,3,Why can not Hindu women be the soldier of Hind...
4,4,How would you write out twelve lakh twelve tho...
...,...,...
5369,5369,Why do we have two eyes?\n
5370,5370,What role does music play in our life?\n
5371,5371,Which is the best coaching for MP psc?\n
5372,5372,Which mail server is used for messaging in AT&...


In [5]:
train.duplicated().unique()

array([False])

In [6]:
test.duplicated().unique()

array([False])

# **Cosine Similarity**

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf_vectorizer = TfidfVectorizer(max_features = 8000, stop_words = 'english')
tfidf_vectorizer.fit(train['Content'])
tfidf_vect_content = tfidf_vectorizer.transform(train['Content'])

In [9]:
tfidf_vectorizer_test = TfidfVectorizer(max_features = 8000, stop_words = 'english')
tfidf_vectorizer_test.fit(test['Content'])
tfidf_vect_content_test = tfidf_vectorizer.transform(test['Content'])

In [ ]:
duplicates_cos = 0
start_time = time()
for row in tfidf_vect_content_test:
  cos_sim = cosine_similarity(row, tfidf_vect_content)
  scores = cos_sim[0]
  duplicates_cos += len(scores[scores>0.8])
end_time = time()

elapsed_cos = end_time - start_time
print("Number of duplicates with Exact Cosine: ", duplicates_cos)
print("Execution time for Exact Cosine: ", str(elapsed_cos))

Number of duplicates with Exact Cosine:  1908
Execution time for Exact Cosine:  333.0595178604126


# **LSH Cosine**

In [ ]:
from collections import defaultdict
from scipy.sparse import vstack

def train_lsh(X_tfidf, n_vectors, seed=None):    
    if seed is not None:
        np.random.seed(seed)

    dim = X_tfidf.shape[1]
    random_vectors = np.random.randn(dim, n_vectors)  

    # partition data points into bins,
    # and encode bin index bits into integers
    bin_indices_bits = X_tfidf.dot(random_vectors) >= 0
    powers_of_two = 1 << np.arange(n_vectors - 1, -1, step=-1)
    bin_indices = bin_indices_bits.dot(powers_of_two)

    # update `table` so that `table[i]` is the list of document ids with bin index equal to i
    table = defaultdict(list)
    for idx, bin_index in enumerate(bin_indices):
        table[bin_index].append(idx)
    bin_size = len(table.keys())
    matrices = [None] * bin_size

    # create new sparse matrices for each bean
    for key in table:
      items_in_a_bin = table[key]
      # print(len(items_in_a_bin))
      matrix = X_tfidf[items_in_a_bin[0]]
      # print(matrix)
      for t in items_in_a_bin[1:]:
        matrix = vstack([matrix, X_tfidf[t]])
      matrices[key] = matrix

    # note that we're storing the bin_indices here
    # so we can do some ad-hoc checking with it,
    # this isn't actually required
    model = {'table': table,
             'n_vectors': n_vectors,
             'random_vectors': random_vectors,
             'bin_indices': bin_indices,
             'bin_indices_bits': bin_indices_bits,
             'matrices': matrices}

    return model

In [ ]:
def get_duplicates_lsh(X_tfidf, model, threshold):  

  duplicates = 0

  for row in X_tfidf:
    bin_indices_bits = row.dot(model['random_vectors']) >= 0
    powers_of_two = 1 << np.arange(model['n_vectors'] - 1, -1, step=-1)
    bin_indices = bin_indices_bits.dot(powers_of_two)[0]
    cos_sim = cosine_similarity(row, model['matrices'][bin_indices])
    scores = cos_sim[0]
    duplicates += len(scores[scores>threshold])
  return duplicates

In [ ]:
LSH_models = [None]*10
for k in range(1, 11):
  n_vectors = k
  start_time = time()
  LSH_models[k-1] = train_lsh(tfidf_vect_content, n_vectors, seed=143)
  end_time = time()
  build_time_LSHcos = end_time - start_time
  print("LSH-Cosine build time with parameter K =", k, "is", build_time_LSHcos, "\n")
  LSH_models.append(LSHCosine)

LSH-Cosine build time with parameter K = 1 is 880.6421494483948 

LSH-Cosine build time with parameter K = 2 is 460.0274920463562 

LSH-Cosine build time with parameter K = 3 is 288.79246187210083 

LSH-Cosine build time with parameter K = 4 is 197.69322443008423 

LSH-Cosine build time with parameter K = 5 is 142.4743993282318 

LSH-Cosine build time with parameter K = 6 is 110.5415632724762 

LSH-Cosine build time with parameter K = 7 is 101.41950559616089 

LSH-Cosine build time with parameter K = 8 is 98.83390665054321 

LSH-Cosine build time with parameter K = 9 is 98.75844717025757 

LSH-Cosine build time with parameter K = 10 is 98.37308835983276 



In [ ]:
threshold = 0.8

for k in range(1, 11):
  start_time = time()
  duplicates_LSHcos = get_duplicates_lsh(tfidf_vect_content_test, LSH_models[k-1], threshold)
  end_time = time()
  query_time_LSHcos = end_time - start_time
  print("LSH query time with parameter K =", k, "is", query_time_LSHcos, "and the number duplicates =", duplicates_LSHcos, "\n")

LSH query time with parameter K = 1 is 196.07279872894287 and the number duplicates = 1630 

LSH query time with parameter K = 2 is 91.49265551567078 and the number duplicates = 1552 

LSH query time with parameter K = 3 is 42.14195442199707 and the number duplicates = 1387 

LSH query time with parameter K = 4 is 21.633859634399414 and the number duplicates = 1211 

LSH query time with parameter K = 5 is 12.589418411254883 and the number duplicates = 888 

LSH query time with parameter K = 6 is 8.332197189331055 and the number duplicates = 944 

LSH query time with parameter K = 7 is 6.028832674026489 and the number duplicates = 832 

LSH query time with parameter K = 8 is 4.802473306655884 and the number duplicates = 720 

LSH query time with parameter K = 9 is 4.349105358123779 and the number duplicates = 679 

LSH query time with parameter K = 10 is 4.127082824707031 and the number duplicates = 656 



# **Jaccard similarity**

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Data preprocessing**

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def data_preprocessing(text):
  #Split into Words
  tokens = word_tokenize(text)

  #Lowercase
  words = [w.lower() for w in tokens]

  #Filter Out Punctuation
  words = [word for word in words if word.isalnum()]

  #Filter out Stop Words
  words = [w for w in words if not w in stop_words]
  
  return words

In [ ]:
train_content = train["Content"]
test_content = test["Content"]

In [ ]:
preprocessed_test, preprocessed_train = [], []
for v in train_content:
  prep_v = data_preprocessing(v)
  if(len(prep_v) > 0):
    preprocessed_train.append(prep_v)

for t in test_content:
  prep_t = data_preprocessing(t)
  if(len(prep_t) > 0):
    preprocessed_test.append(prep_t)

**Finding the number of duplicates using Jaccard Similarity**

In [ ]:
def jaccard_similarity(set1, set2): 

    set1, set2 = set(set1), set(set2)
    # Find the intersection
    intersection = set1.intersection(set2)
    # Find the union
    union = set1.union(set2)
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [ ]:
from time import time

duplicates_jac = 0

start_time = time()

for x1 in preprocessed_test:
  for x2 in preprocessed_train:
    jak_sim = jaccard_similarity(x1, x2)
    if(jak_sim > 0.8):
      duplicates_jac += 1

end_time = time()
print("Number of duplicates with Exact-Jaccard: ", duplicates_jac)
elapsed_jac = end_time-start_time
print("Execution time with Exact-Jaccard: ", str(elapsed_jac))

Number of duplicates with Exact-Jaccard:  2191
Execution time with Exact-Jaccard:  5321.060403108597


# Min-Hash Jaccard

In [ ]:
train_content = train["Content"]
test_content = test["Content"]

In [ ]:
!pip install datasketch

     |████████████████████████████████| 71kB 4.9MB/s 


In [ ]:
def build_MinHashLSH(lsh, permutations):
  minhash = []

  for v in train_content:
    x = data_preprocessing(v)
    s = set(x)
    m = MinHash(num_perm=permutations)
    for d in s:
      m.update(d.encode('utf8'))
    minhash.append(m)

  for i,m in enumerate(minhash):
    lsh.insert(i,m)

  return lsh

In [ ]:
def find_dupliactes(lsh, permutations):
  dup = 0

  for t in test_content:
    x = data_preprocessing(t)
    s = set(x)
    m = MinHash(num_perm=permutations)
    for d in s:
      m.update(d.encode('utf8'))
    result = lsh.query(m)
    dup += len(result)

  return dup

In [ ]:
from datasketch import MinHash, MinHashLSH
duplicates = []
for p in [16, 32, 64]:
  lsh = MinHashLSH(threshold=0.8, num_perm=p)
  
  start_time = time()
  lsh = build_MinHashLSH(lsh, p)
  end_time = time()
  build_time_LSHjac = end_time - start_time
  print("LSH-Jaccard build time with the number of permutations p =", p, "is", build_time_LSHjac, "\n")

  start_time = time()
  duplicates_LSHjac = find_dupliactes(lsh, p)
  end_time = time()
  query_time_LSHjac = end_time - start_time
  print("LSH-Jaccard query time with the number of permutations p =", p, "is", query_time_LSHjac, "and the number duplicates =", duplicates_LSHjac, "\n")

LSH-Jaccard build time with the number of permutations p = 16 is 423.3177146911621 

LSH-Jaccard query time with the number of permutations p = 16 is 4.397579669952393 and the number duplicates = 8547 

LSH-Jaccard build time with the number of permutations p = 32 is 489.6985549926758 

LSH-Jaccard query time with the number of permutations p = 32 is 4.9695165157318115 and the number duplicates = 7778 

LSH-Jaccard build time with the number of permutations p = 64 is 625.1319687366486 

LSH-Jaccard query time with the number of permutations p = 64 is 6.382096529006958 and the number duplicates = 7910 

